In [114]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import os
import torch
from torch import nn
from torch.utils.data import Dataset, TensorDataset, DataLoader # for dealing with data
from torchvision import datasets, transforms
from torchvision import models
from torchsummary import summary
import torch.optim as optim # Optimization package

In [218]:
nswre74_control = pd.read_excel('/Users/mawuliagamah/gitprojects/causal_inference/causal_inference/datasets/nswre74_control.xls')
nswre74_treated = pd.read_excel('/Users/mawuliagamah/gitprojects/causal_inference/causal_inference/nswre74_treated.xlsx')
nswre74 = pd.concat([nswre74_control,nswre74_treated])


nswre74_shuffled = nswre74.sample(frac=1)
nswre74_shuffled.shape
# Train and test split
nswre74_train = nswre74_shuffled.iloc[0:356]
nswre74_test = nswre74_shuffled.iloc[357:len(nswre74)]

In [219]:
input_dim = 9 # Variables within dataset 
hidden_dim = 2 # hidden layers
output_dim= 2    # number of classes


# Convert features to tensor 
age_array = np.asarray(nswre74_train['age'])
education_array= np.asarray(nswre74_train['education'])
black_array = np.asarray(nswre74_train['black'])
hispanic_array = np.asarray(nswre74_train['hispanic'])
nodegree_array = np.asarray(nswre74_train['nodegree'])
re74_array = np.asarray(nswre74_train['re74'])
re75_array = np.asarray(nswre74_train['re75'])
re78_array = np.asarray(nswre74_train['re78'])

features_arrray = [age_array,education_array,black_array,hispanic_array,nodegree_array,re74_array,re75_array,re78_array]
features_arrray = np.transpose(features_arrray)
X = torch.Tensor(features_arrray)

#Convert target to tensor 
targets = nswre74_train['treat']
Y = torch.tensor(targets.values)

flatten = nn.Flatten()
X = flatten(X)




In [227]:
#Define neural netowrk class 
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            #Neural net input layer ,  356 input layers, to 15 hidden neurons
            nn.Linear(8,356),
            #relu activation function for hidden layer 1
            nn.ReLU(),
            #relu activation function for hidden layer 2
            nn.Linear(356, 356),
            nn.ReLU(),
            #Output layer 
            #nn.Linear(356, 2),
            nn.Softmax(dim=1)
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


model = NeuralNetwork() #Initialise model


In [221]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")
model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=8, out_features=356, bias=True)
    (1): ReLU()
    (2): Linear(in_features=356, out_features=356, bias=True)
    (3): ReLU()
    (4): Linear(in_features=356, out_features=1, bias=True)
  )
)


##     Training the model

In [228]:
train = TensorDataset(X, Y)
trainloader = DataLoader(train, batch_size=1, shuffle=False)

number_of_epochs = 3
learning_rate = 1e-5
max_iterations = 200

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.LBFGS(model.parameters(),  lr=learning_rate , max_iter=max_iterations ) 

#optimizer = optim.SGD(net.parameters(), lr=LEARNING_RATE) # slight difference: we optimize w.r.t. the net parameters now


In [229]:
#Training loop

for epoch in range(number_of_epochs):
    train_loader_iter = iter(trainloader)
    for batch_idx, (inputs, labels) in enumerate(train_loader_iter):
       
        # Zero gradients for batch 
        model.zero_grad() 
        # Every data instance is an input + label pair
        inputs, labels = X, Y
        # Make predictions for given batch 
        output = model(inputs)
        # Compute the loss and its gradients
        loss = loss_fn(output, labels)
        loss.backward()
        # Adjust learning weights
        optimizer.step()
        # Gather data and report
        print("----------------------------------------")
        print("Output (UPDATE: Epoch #" + str(epoch + 1) + ", Batch #" + str(batch_idx + 1) + "):")
        #print(net(X)) # much better!

print("----------------------------------------")

TypeError: step() missing 1 required positional argument: 'closure'

In [208]:
model_predictions = model(X)

In [214]:
output_df = pd.DataFrame(output.detach().numpy())

<p> cps1 match </p>

In [215]:
output_df

,0,1,2
0,0.650093,-0.186797,-1.562804
1,49.975693,49.208115,-101.912224
2,0.608929,-0.178376,-1.435403
3,43.270592,42.628761,-88.470245
4,47.578587,46.778503,-96.860901
...,...,...,...
351,0.510414,-0.128720,-1.459245
352,0.712492,-0.204728,-1.707634
353,40.331329,39.695103,-82.205582
354,1.220890,-0.427712,-2.055882
